In [1]:
import copy
import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from modules import solver, presolver
from pyballistics import ozvb_lagrange, get_powder_names
from tqdm import tqdm
from joblib import Parallel, delayed

## Подгрузка начальных точек
формируется таблица с начальными точками для каждого пороха

In [2]:
tabel_init_points = pd.read_csv("data/init_points.csv")
tabel_init_points.loc[tabel_init_points['success']==False]

,dbname,wq,ro,reason,success


## Решение обратной задачи

In [3]:
core = joblib.cpu_count() - 2

In [5]:
def step(row):
    initial_dict = presolver.init_dict(row['dbname'], row['wq'], row['ro'])
    solv = solver.Solver(initial_dict)
    solv.solution()
    return solv.get_solution()   

In [ ]:
%%time
tabel_result = Parallel(n_jobs=core, verbose=10)(delayed(step)(row) for i, row in tabel_init_points.iterrows())

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   37.6s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   47.5s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:  2.1min


In [5]:
result = copy.deepcopy(tabel_result)

In [6]:
table = list()
for row in result:
    if row is None:
        continue
    table.append(row)

In [7]:
table = pd.DataFrame(table)

In [8]:
table.to_csv("data/table_result_lenght.csv", index=False)

## Обработка

In [12]:
table.head()

,name,succes,wq,ro,mass,reason
0,ДРП,True,3.869251,1011.404983,2650.941081,x_p
1,КЗДП,True,3.869251,1011.404979,2650.945574,x_p
2,ВУ фл,True,1.941803,372.088906,1889.718752,x_p
3,СФ 033,True,2.105988,351.792992,1801.638582,x_p
4,ВТ,True,2.009104,375.303381,1859.456452,x_p


In [31]:
table.sort_values(by=['mass']).head()

,name,succes,wq,ro,mass,reason
101,УГ-2,True,0.866869,89.828520,300.690783,x_p
41,14/1 тр В/А,True,1.425524,802.135467,347.555869,x_p
46,15/1 тр В/А,True,1.517220,844.921773,358.608036,x_p
43,12/1 тр МН,True,1.539647,854.173998,378.514633,x_p
36,12/7 В/А,True,1.477258,864.207748,378.976982,x_p
45,ДГ-4 13/1,True,1.500202,837.135182,380.383730,x_p
38,12/7,True,1.645166,911.719247,382.280032,x_p
40,14/7 В/А,True,1.627366,945.833541,383.870878,x_p
63,МАП-1 23/1,True,1.620367,886.219706,392.891182,x_p
42,12/1 Тр,True,2.025696,600.943886,397.829038,x_p


In [28]:
res = ozvb_lagrange(presolver.init_dict("14/1 тр В/А", 1.425524, 802.135467))

In [30]:
res['layers'][-1]['x'][-1]

5.543